In [1]:
import torch
import numpy as np
from botorch.models import SingleTaskGP
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.fit import fit_gpytorch_model
from botorch.acquisition.multi_objective import qExpectedHypervolumeImprovement
from botorch.optim import optimize_acqf
from botorch.utils.multi_objective.box_decompositions import NondominatedPartitioning
from botorch.utils.multi_objective.pareto import is_non_dominated
from botorch.utils import standardize
from botorch.acquisition.multi_objective.objective import IdentityMCMultiOutputObjective
from sklearn.preprocessing import MinMaxScaler

c:\Users\zopev1\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
paramConfig = {
    'c1': {'lowerBound': 0, 'upperBound': 1, 'exponent': 1.0, 'name': 'W', 'unit': 'dimensionless', 'type': 'hardening'}, 
    'c2': {'lowerBound': 0, 'upperBound': 2, 'exponent': 1000.0, 'name': 'K', 'unit': 'MPa', 'type': 'yielding'}, 
    'c3': {'lowerBound': 0, 'upperBound': 1, 'exponent': 0.1, 'name': 'e0', 'unit': 'dimensionless', 'type': 'hardening'}, 
    'c4': {'lowerBound': 0, 'upperBound': 1, 'exponent': 1.0, 'name': 'n', 'unit': 'dimensionless', 'type': 'hardening'}, 
    'c5': {'lowerBound': 0, 'upperBound': 2, 'exponent': 1000.0, 'name': 'sigma_y', 'unit': 'MPa', 'type': 'yielding'}, 
    'c6': {'lowerBound': 0, 'upperBound': 1, 'exponent': 1000.0, 'name': 'sigma_sat', 'unit': 'MPa', 'type': 'hardening'}, 
    'c7': {'lowerBound': 0, 'upperBound': 1, 'exponent': 1000.0, 'name': 'b', 'unit': 'dimensionless', 'type': 'hardening'}
}

geometries = ['NDBR50', 'NDBR6', 'CHD6']


In [3]:
# Load the data
combined_interpolated_param_to_geom_FD_Curves_smooth = np.load('combined_interpolated_param_to_geom_FD_Curves_smooth.npy', allow_pickle=True).tolist()
targetCurves = np.load('targetCurves.npy', allow_pickle=True).tolist()

# Define the function for the RMSE loss
def lossFD(y, y_hat):
    return torch.sqrt(torch.mean((y - y_hat)**2))

# Calculate losses and prepare data for model
params = []
losses = []
for param_tuple, geom_dict in combined_interpolated_param_to_geom_FD_Curves_smooth.items():
    params.append([value for param, value in param_tuple])
    losses.append([lossFD(torch.tensor(geom_dict[geometry]["force"]), torch.tensor(targetCurves[geometry]["force"])) for geometry in geometries])

# Convert your data to the tensor(float 64)
X = torch.tensor(params, dtype=torch.float64)
Y = torch.stack([torch.tensor(loss, dtype=torch.float64) for loss in losses])

scaler = MinMaxScaler()
X_normalized = torch.tensor(scaler.fit_transform(X.numpy()), dtype=torch.float64)

# Standardize Y to have zero mean and unit variance
Y_standardized = standardize(Y)

# Define the bounds of the search space
lower_bounds = torch.tensor([paramConfig[param]['lowerBound'] for param in paramConfig.keys()]).float()
upper_bounds = torch.tensor([paramConfig[param]['upperBound'] for param in paramConfig.keys()]).float()
#normalise the bounds in accordance to the normalised params
bounds = torch.tensor([[0.0]*X_normalized.shape[1], [1.0]*X_normalized.shape[1]])

# Initialize model
model = SingleTaskGP(X_normalized, Y_standardized)
mll = ExactMarginalLogLikelihood(model.likelihood, model)
fit_gpytorch_model(mll)

# Define the acquisition function
ref_point = Y_standardized.max(dim=0).values - 0.01

partitioning = NondominatedPartitioning(ref_point=ref_point, Y=Y_standardized)
acq_func = qExpectedHypervolumeImprovement(
    model=model,
    partitioning=partitioning,
    ref_point=ref_point,
    objective=IdentityMCMultiOutputObjective(),
)


# Optimize the acquisition function
candidates, _ = optimize_acqf(
    acq_function=acq_func,
    bounds=bounds,
    q=1,#q: This is the number of points to sample in each step
    num_restarts=10,#num_restarts: This is the number of starting points for the optimization.
    raw_samples=1000,#raw_samples: This is the number of samples to draw when initializing the optimization
)

# Unnormalize the candidates
candidates = candidates * (X.max(dim=0).values - X.min(dim=0).values) + X.min(dim=0).values

#converting to dictionary
next_param_dicts = [{param: value.item() for param, value in zip(paramConfig.keys(), next_param)} for next_param in candidates]

print(next_param_dicts)

[{'c1': 0.02144253413953988, 'c2': 1029.3836305231819, 'c3': 0.04639085410225211, 'c4': 0.4592792981981647, 'c5': 1990.7990799079907, 'c6': 995.5995599559956, 'c7': 162.59976643713156}]
